In [6]:
# ==============================================================
# OPTIMAL STOPPING TRADING STRATEGY WITH DYNAMIC BOUNDARIES PLOT
# ==============================================================
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go
from plotly.subplots import make_subplots
warnings.filterwarnings('ignore')

# Plotting style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# ==============================================================
# 1. DATA DOWNLOAD
# ==============================================================
def download_stock_data(tickers, start_date, end_date, interval='1m'):
    """Download stock data for multiple tickers."""
    data = {}
    for ticker in tickers:
        try:
            df = yf.download(ticker, start=start_date, end=end_date, interval=interval, progress=False)
            if not df.empty:
                data[ticker] = df['Close']
                print(f"✓ Downloaded {ticker}: {len(df)} rows")
        except Exception as e:
            print(f"✗ Failed to download {ticker}: {e}")
    return data

# Example tickers (меньше для тестирования)
tickers = [
    "AAPL","MSFT","GOOGL","AMZN","META",
    "NVDA", "BRK-B","JPM","V",
    "JNJ","WMT","PG", "HD",
    "BAC","DIS","ADBE","PFE","NFLX"
]

end_date = datetime.now()
start_date = end_date - timedelta(days=8)  # 10 дня данных

print("=" * 60)
print("DOWNLOADING STOCK DATA")
print("=" * 60)
price_data = download_stock_data(tickers, start_date, end_date, interval='1m')

# ==============================================================
# 2. GBM PARAMETER ESTIMATION
# ==============================================================
def estimate_gbm_params(price_series, window=300):
    """Estimate mu and sigma for GBM from log returns."""
    if len(price_series) < 2:
        return None, None
    
    # Используем скользящее окно
    prices = price_series[-min(window, len(price_series)):].values
    log_returns = np.log(prices[1:] / prices[:-1])
    
    if len(log_returns) < 2:
        return None, None
    
    # Предполагаем минутные данные
    dt = 1/252/6.5/60  # 1 минута в годах
    mu_hat = np.mean(log_returns) * dt + 0.5 * np.var(log_returns) * dt
    sigma_hat = np.sqrt(np.var(log_returns) * dt)
    
    return float(mu_hat), float(sigma_hat)

# ==============================================================
# 3. REAL OPTION MODEL
# ==============================================================
class RealOptionModel:
    """Real Option Problem with GBM dynamics."""
    
    def __init__(self, mu=0.2, sigma=0.2, rho=0.5, kappa=5.0, theta=0.5, lam=0.1):
        if rho <= mu:
            raise ValueError(f"Discount rate ρ ({rho}) must be > drift μ ({mu})")
        if not 0 < theta < 1:
            raise ValueError(f"Theta ({theta}) must be in (0, 1)")
        
        self.mu = mu
        self.sigma = sigma
        self.rho = rho
        self.kappa = kappa
        self.theta = theta
        self.lam = lam
        
        # Compute characteristic roots
        self.alpha_plus, self.alpha_minus = self._compute_alpha_roots()
        
        # Compute P coefficient
        self.P = self._compute_P_coefficient()
        
        self.y_lambda = np.exp(-1 - kappa * rho / lam)
    
    def _compute_alpha_roots(self):
        """Compute α+ and α-."""
        a = 0.5 * self.sigma**2
        b = self.mu - 0.5 * self.sigma**2
        c = -self.rho
        discriminant = b**2 - 4*a*c
        alpha_plus = (-b + np.sqrt(discriminant)) / (2*a)
        alpha_minus = (-b - np.sqrt(discriminant)) / (2*a)
        return alpha_plus, alpha_minus
    
    def _compute_P_coefficient(self):
        """Compute P coefficient for H_pi."""
        denom = self.rho - self.theta * (self.mu + 0.5 * self.sigma**2 * (self.theta - 1))
        if denom <= 0:
            raise ValueError("Parameters lead to unbounded H_pi")
        return 1.0 / denom
    
    def b_star_classical(self):
        """Classical optimal stopping boundary."""
        coef = -self.alpha_minus / (self.theta - self.alpha_minus)
        return (coef * self.kappa / self.P) ** (1.0 / self.theta)
    
    def update_parameters(self, mu, sigma):
        """Update model parameters dynamically."""
        self.mu = mu
        self.sigma = sigma
        self.alpha_plus, self.alpha_minus = self._compute_alpha_roots()
        self.P = self._compute_P_coefficient()

# ==============================================================
# 4. DYNAMIC OPTIMAL STOPPING STRATEGY WITH BOUNDARY TRACKING
# ==============================================================
class DynamicOptimalStoppingTrader:
    """Implements dynamic optimal stopping strategy with boundary tracking."""
    
    def __init__(self, rho=0.5, kappa=5.0, theta=0.5, lam=0.1):
        self.rho = rho
        self.kappa = kappa
        self.theta = theta
        self.lam = lam
        
        # Current model and state
        self.model = None
        self.b_star = None
        self.a_star = None
        self.reference_price = None
        
        # Trackers
        self.ticks_since_last_action = 0
        self.ticks_since_last_recalc = 0
        self.signals = []
        self.profits = []
        
        # NEW: Track boundaries at each step
        self.boundary_history = {
            'timestamps': [],
            'prices': [],
            'upper_boundary': [],  # b_star * reference_price
            'lower_boundary': [],  # a_star * reference_price
            'b_star_values': [],   # b_star (relative)
            'a_star_values': []    # a_star (relative)
        }
    
    def initialize_model(self, mu, sigma, initial_price, timestamp):
        """Initialize model with initial parameters."""
        self.model = RealOptionModel(
            mu=mu, sigma=sigma, rho=self.rho,
            kappa=self.kappa, theta=self.theta, lam=self.lam
        )
        self.reference_price = initial_price
        self._update_boundaries()
        
        # Record initial boundaries
        self._record_boundaries(timestamp, initial_price)
    
    def _update_boundaries(self):
        """Update buy/sell boundaries based on current model."""
        if self.model:
            b_raw = self.model.b_star_classical()
            self.b_star = np.clip(b_raw, 0.999, 1.001)
            self.a_star = 1.0 / self.b_star
    
    def _record_boundaries(self, timestamp, current_price):
        """Record current boundaries."""
        if self.b_star is not None and self.reference_price is not None:
            self.boundary_history['timestamps'].append(timestamp)
            self.boundary_history['prices'].append(current_price)
            self.boundary_history['upper_boundary'].append(self.reference_price * self.b_star)
            self.boundary_history['lower_boundary'].append(self.reference_price * self.a_star)
            self.boundary_history['b_star_values'].append(self.b_star)
            self.boundary_history['a_star_values'].append(self.a_star)
    
    def update_model_parameters(self, mu, sigma, current_price, timestamp):
        """Update model parameters and reset reference."""
        if self.model:
            self.model.update_parameters(mu, sigma)
            self.reference_price = current_price
            self._update_boundaries()
            self.ticks_since_last_recalc = 0
            
            # Record boundaries after update
            self._record_boundaries(timestamp, current_price)
    
    def get_normalized_price(self, current_price):
        """Normalize price: current_price / reference_price."""
        if self.reference_price is None or self.reference_price == 0:
            return 1.0
        return current_price / self.reference_price
    
    def process_tick(self, timestamp, price, price_history):
        """Process a single tick and return action."""
        # Извлекаем скалярное значение из Series
        if hasattr(price, 'iloc'):
            current_price = float(price.iloc[0] if len(price) == 1 else price.values[0])
        else:
            current_price = float(price)
        
        # Initialize model if not done yet
        if self.model is None or self.reference_price is None:
            if len(price_history) >= 100:
                mu, sigma = estimate_gbm_params(price_history[:100])
                if mu is not None and sigma is not None:
                    self.initialize_model(mu, sigma, current_price, timestamp)
                    return 'INIT', current_price, 1.0
            return 'HOLD', current_price, None
        
        # Calculate normalized price
        ratio = self.get_normalized_price(current_price)
        
        # Increment counters
        self.ticks_since_last_action += 1
        self.ticks_since_last_recalc += 1
        
        action = 'HOLD'
        
        # Check for SELL signal (crossing b_star from below)
        if ratio >= self.b_star:
            action = 'SELL'
            self.signals.append((timestamp, current_price, 'SELL'))
            self.ticks_since_last_action = 0
            
            # После продажи пересчитываем параметры
            if len(price_history) >= 100:
                mu, sigma = estimate_gbm_params(price_history[-3000:])
                if mu is not None and sigma is not None:
                    self.update_model_parameters(mu, sigma, current_price, timestamp)
        
        # Check for BUY signal (crossing a_star from above)
        elif ratio <= self.a_star:
            action = 'BUY'
            self.signals.append((timestamp, current_price, 'BUY'))
            self.ticks_since_last_action = 0
            
            # После покупки пересчитываем параметры
            if len(price_history) >= 100:
                mu, sigma = estimate_gbm_params(price_history[-100:])
                if mu is not None and sigma is not None:
                    self.update_model_parameters(mu, sigma, current_price, timestamp)
        
        # Если прошло 100 тиков без действий - пересчитываем параметры
        elif self.ticks_since_last_recalc >= 100 and len(price_history) >= 100:
            mu, sigma = estimate_gbm_params(price_history[-100:])
            if mu is not None and sigma is not None:
                self.update_model_parameters(mu, sigma, current_price, timestamp)
        
        # Record boundaries for this tick (если не было обновления)
        if action == 'HOLD' and len(self.boundary_history['timestamps']) == 0 or \
           self.boundary_history['timestamps'][-1] != timestamp:
            self._record_boundaries(timestamp, current_price)
        
        return action, current_price, ratio
    
    def calculate_profits(self):
        """Calculate cumulative profit from signals."""
        if not self.signals:
            return []
        
        cumulative = 0
        profits = []
        df = pd.DataFrame(self.signals)
        b = 0
        s = 0
        
        for timestamp, price, action in self.signals:
            price_val = float(price)
            
            if action == 'BUY':
                cumulative -= price_val
                b += 1
            elif action == 'SELL':
                cumulative += price_val
                s +=1
            
            profits.append((timestamp, cumulative))
        
        cumulative += -self.boundary_history['prices'][-1] * (s - b)
        profits.append((self.boundary_history['timestamps'][-1], cumulative))
        self.profits = profits
        return profits
    
    def get_summary(self):
        """Get trading summary."""
        num_signals = len(self.signals)
        final_profit = self.profits[-1][1] if self.profits else 0
        return {
            'num_signals': num_signals,
            'final_profit': final_profit,
            'current_b_star': self.b_star,
            'current_a_star': self.a_star,
            'boundary_history': self.boundary_history
        }

# ==============================================================
# 5. DASHBOARD WITH DYNAMIC BOUNDARIES
# ==============================================================
class DynamicTradingDashboard:
    """Creates dashboard with price and dynamic boundaries."""
    
    def plot_stock_analysis(self, ticker, price_series, trader):
        """Create comprehensive plot for a single stock."""
        fig, axes = plt.subplots(4, 1, figsize=(16, 14), 
                                 gridspec_kw={'height_ratios': [3, 2, 1, 1]})
        
        # ============================================
        # 1. PRICE WITH DYNAMIC BOUNDARIES AND SIGNALS
        # ============================================
        ax1 = axes[0]
        
        # Plot price
        ax1.plot(price_series.index, price_series.values, 
                'b-', linewidth=1, alpha=0.7, label='Price')
        
        # Plot dynamic boundaries if available
        if trader.boundary_history['timestamps']:
            # Convert to pandas Series for easier plotting
            timestamps = pd.to_datetime(trader.boundary_history['timestamps'])
            upper_boundary = pd.Series(trader.boundary_history['upper_boundary'], index=timestamps)
            lower_boundary = pd.Series(trader.boundary_history['lower_boundary'], index=timestamps)
            
            # Plot boundaries with different styles to show dynamics
            ax1.plot(upper_boundary.index, upper_boundary.values, 
                    'r--', linewidth=1.5, alpha=0.6, label='Upper (Sell) Boundary')
            ax1.plot(lower_boundary.index, lower_boundary.values, 
                    'g--', linewidth=1.5, alpha=0.6, label='Lower (Buy) Boundary')
            
            # Fill between boundaries
            ax1.fill_between(upper_boundary.index, 
                           lower_boundary.values, 
                           upper_boundary.values, 
                           color='yellow', alpha=0.1, label='Trading Zone')
        
        # Extract signal points
        buy_signals = [(t, p) for t, p, a in trader.signals if a == 'BUY']
        sell_signals = [(t, p) for t, p, a in trader.signals if a == 'SELL']
        
        if buy_signals:
            buy_times, buy_prices = zip(*buy_signals)
            ax1.scatter(buy_times, buy_prices, color='green', 
                       marker='*', s=200, zorder=10, label='BUY Signal', alpha=0.9)
        
        if sell_signals:
            sell_times, sell_prices = zip(*sell_signals)
            ax1.scatter(sell_times, sell_prices, color='red', 
                       marker='.', s=150, zorder=10, label='SELL Signal', alpha=0.9)
        
        ax1.set_title(f'{ticker} - Price with Dynamic Optimal Stopping Boundaries', fontsize=14)
        ax1.set_ylabel('Price ($)', fontsize=12)
        ax1.legend(loc='best', fontsize=9)
        ax1.grid(True, alpha=0.3)
        
                
        # ============================================
        # 2. CUMULATIVE PROFIT
        # ============================================
        ax3 = axes[2]
        if trader.profits:
            profit_times, profit_values = zip(*trader.profits)
            ax3.plot(profit_times, profit_values, 'purple', linewidth=2, 
                    label='Cumulative Profit')
            ax3.axhline(y=0, color='k', linestyle='-', alpha=0.3)
            
            # Mark buy/sell points
            buy_profits = [(t, p) for (t, p), (_, _, a) in zip(trader.profits, trader.signals) if a == 'BUY']
            sell_profits = [(t, p) for (t, p), (_, _, a) in zip(trader.profits, trader.signals) if a == 'SELL']
            
            if buy_profits:
                buy_t, buy_p = zip(*buy_profits)
                ax3.scatter(buy_t, buy_p, color='green', marker='*', s=100, zorder=5)
            
            if sell_profits:
                sell_t, sell_p = zip(*sell_profits)
                ax3.scatter(sell_t, sell_p, color='red', marker='.', s=80, zorder=5)
        
        ax3.set_title('Cumulative Profit', fontsize=12)
        ax3.set_ylabel('Profit ($)', fontsize=12)
        ax3.legend(loc='best')
        ax3.grid(True, alpha=0.3)
        
        
        plt.tight_layout()
        return fig
    
    def plot_boundary_summary(self, ticker, trader):
        """Create summary plot of boundary behavior."""
        if not trader.boundary_history['timestamps']:
            return None
        
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        
        timestamps = pd.to_datetime(trader.boundary_history['timestamps'])
        b_star_vals = trader.boundary_history['b_star_values']
        a_star_vals = trader.boundary_history['a_star_values']
        
        # 1. Boundary distribution
        ax1 = axes[0, 0]
        ax1.hist(b_star_vals, bins=20, alpha=0.6, color='red', label='Sell Boundary')
        ax1.hist(a_star_vals, bins=20, alpha=0.6, color='green', label='Buy Boundary')
        ax1.set_title('Boundary Value Distribution')
        ax1.set_xlabel('Boundary Value')
        ax1.set_ylabel('Frequency')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # 2. Boundary spread
        ax2 = axes[0, 1]
        spread = np.array(b_star_vals) - np.array(a_star_vals)
        ax2.plot(timestamps, spread, 'b-', alpha=0.7)
        ax2.set_title('Boundary Spread (b* - a*)')
        ax2.set_xlabel('Time')
        ax2.set_ylabel('Spread')
        ax2.grid(True, alpha=0.3)
        
        # 3. Boundary changes
        ax3 = axes[1, 0]
        b_star_changes = np.abs(np.diff(b_star_vals))
        a_star_changes = np.abs(np.diff(a_star_vals))
        ax3.plot(timestamps[1:], b_star_changes, 'r-', alpha=0.6, label='Δ b*')
        ax3.plot(timestamps[1:], a_star_changes, 'g-', alpha=0.6, label='Δ a*')
        ax3.set_title('Boundary Changes Magnitude')
        ax3.set_xlabel('Time')
        ax3.set_ylabel('Absolute Change')
        ax3.legend()
        ax3.grid(True, alpha=0.3)
        
        # 4. Signal locations relative to boundaries
        ax4 = axes[1, 1]
        if trader.signals:
            # Calculate boundary values at signal times
            signal_boundaries = []
            signal_types = []
            
            for sig_time, sig_price, sig_action in trader.signals:
                # Find closest boundary record
                idx = min(range(len(timestamps)), 
                         key=lambda i: abs(timestamps[i] - pd.Timestamp(sig_time)))
                if idx < len(b_star_vals):
                    signal_boundaries.append(b_star_vals[idx] if sig_action == 'SELL' else a_star_vals[idx])
                    signal_types.append(sig_action)
            
            if signal_boundaries:
                colors = ['green' if t == 'BUY' else 'red' for t in signal_types]
                ax4.scatter(range(len(signal_boundaries)), signal_boundaries, 
                          c=colors, alpha=0.6)
                ax4.set_title('Boundary Values at Signal Times')
                ax4.set_xlabel('Signal Number')
                ax4.set_ylabel('Boundary Value')
                ax4.grid(True, alpha=0.3)
        
        plt.suptitle(f'{ticker} - Boundary Analysis', fontsize=14, y=1.02)
        plt.tight_layout()
        return fig
    
def extract_data_from_results(results):
    """Извлекает и структурирует данные из results"""
    data = {}
    
    for ticker, ticker_data in results.items():
        trader = ticker_data['trader']
        boundary_history = trader.boundary_history
        signals = ticker_data.get('signals', [])
        profits = ticker_data.get('profits', [])
        summary = ticker_data.get('summary', {})
        
        # Извлекаем временные метки и цены из boundary_history
        timestamps = boundary_history.get('timestamps', [])
        prices = boundary_history.get('prices', [])
        upper_boundary = boundary_history.get('upper_boundary', [])
        lower_boundary = boundary_history.get('lower_boundary', [])
        b_star_values = boundary_history.get('b_star_values', [])
        a_star_values = boundary_history.get('a_star_values', [])
        
        # Извлекаем сигналы
        buy_signals = [(t, p) for t, p, a in signals if a == 'BUY']
        num_buy = len(buy_signals)
        sell_signals = [(t, p) for t, p, a in signals if a == 'SELL']
        num_sell = len(sell_signals)
        
        # Извлекаем прибыли
        profit_times = [t for t, _ in profits]
        profit_values = [p for _, p in profits]
        
        # Извлекаем сводные показатели
        final_profit = summary.get('final_profit', 0)
        num_trades = summary.get('num_signals', 0)
        win_rate = summary.get('win_rate', 0)
        avg_return = summary.get('avg_return_per_trade', 0)
        
        # Сохраняем структурированные данные
        data[ticker] = {
            'timestamps': timestamps,
            'prices': prices,
            'upper_boundary': upper_boundary,
            'lower_boundary': lower_boundary,
            'b_star_values': b_star_values,
            'a_star_values': a_star_values,
            'buy_signals': buy_signals,
            'sell_signals': sell_signals,
            'profit_times': profit_times,
            'profit_values': profit_values,
            'final_profit': final_profit,
            'num_trades': num_trades,
            'num_buy': num_buy,
            'num_sell': num_sell,
            'win_rate': win_rate * 100,  # В процентах
            'avg_return': avg_return,
            'all_signals': signals,
            'all_profits': profits,
            'summary': summary
        }
    
    return data

# ==============================================================
# 2. ФУНКЦИИ ДЛЯ СОЗДАНИЯ ГРАФИКОВ
# ==============================================================
def fig_price_with_boundaries(ticker_data, ticker):
    """График цены с динамическими границами и сигналами"""
    fig = go.Figure()
    
    # Проверяем наличие данных
    if not ticker_data['timestamps']:
        return go.Figure()
    
    # Преобразуем временные метки
    timestamps = pd.to_datetime(ticker_data['timestamps'])
    prices = ticker_data['prices']
    
    # Ценовой ряд
    fig.add_trace(go.Scatter(
        x=timestamps,
        y=prices,
        mode='lines',
        name='Price',
        line=dict(color='#636efa', width=3),
        opacity=0.8
    ))
    
    # Верхняя граница (b_star * reference_price)
    if ticker_data['upper_boundary']:
        fig.add_trace(go.Scatter(
            x=timestamps[:len(ticker_data['upper_boundary'])],
            y=ticker_data['upper_boundary'],
            mode='lines',
            name='Upper (Sell) Boundary',
            line=dict(color='#ef553b', width=2, dash='dash'),
            opacity=0.6
        ))
    
    # Нижняя граница (a_star * reference_price)
    if ticker_data['lower_boundary']:
        fig.add_trace(go.Scatter(
            x=timestamps[:len(ticker_data['lower_boundary'])],
            y=ticker_data['lower_boundary'],
            mode='lines',
            name='Lower (Buy) Boundary',
            line=dict(color='#00cc96', width=2, dash='dash'),
            opacity=0.6
        ))
    
    # Зона торговли (заливка между границами)
    if ticker_data['upper_boundary'] and ticker_data['lower_boundary']:
        min_len = min(len(ticker_data['upper_boundary']), len(ticker_data['lower_boundary']))
        fig.add_trace(go.Scatter(
            x=list(timestamps[:min_len]) + list(reversed(timestamps[:min_len])),
            y=list(ticker_data['upper_boundary'][:min_len]) + list(reversed(ticker_data['lower_boundary'][:min_len])),
            fill='toself',
            fillcolor='rgba(255, 255, 0, 0.1)',
            line=dict(color='rgba(255, 255, 0, 0)'),
            name='Trading Zone',
            showlegend=False
        ))
    
    # Сигналы BUY
    if ticker_data['buy_signals']:
        buy_times, buy_prices = zip(*ticker_data['buy_signals'])
        fig.add_trace(go.Scatter(
            x=pd.to_datetime(buy_times),
            y=buy_prices,
            mode='markers',
            name='BUY Signal',
            marker=dict(
                color='#00cc96',
                symbol='star',
                size=12,
                line=dict(color='white', width=1)
            ),
            hovertemplate='BUY<br>Time: %{x}<br>Price: $%{y:.2f}<extra></extra>'
        ))
    
    # Сигналы SELL
    if ticker_data['sell_signals']:
        sell_times, sell_prices = zip(*ticker_data['sell_signals'])
        fig.add_trace(go.Scatter(
            x=pd.to_datetime(sell_times),
            y=sell_prices,
            mode='markers',
            name='SELL Signal',
            marker=dict(
                color='#ef553b',
                symbol='circle',
                size=10,
                line=dict(color='white', width=1)
            ),
            hovertemplate='SELL<br>Time: %{x}<br>Price: $%{y:.2f}<extra></extra>'
        ))
    
    fig.update_layout(
        title=f'{ticker} - Price with Dynamic Boundaries',
        xaxis_title='Time',
        yaxis_title='Price ($)',
        template='plotly_white',
        height=500,
        hovermode='x unified',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    return fig

def fig_cumulative_profit(ticker_data, ticker):
    """График кумулятивной прибыли"""
    fig = go.Figure()
    
    if ticker_data['profit_times'] and ticker_data['profit_values']:
        fig.add_trace(go.Scatter(
            x=pd.to_datetime(ticker_data['profit_times']),
            y=ticker_data['profit_values'],
            mode='lines+markers',
            name='Cumulative Profit',
            line=dict(color='#764ba2', width=3),
            marker=dict(size=6)
        ))
    
    # Линия нулевой прибыли
    fig.add_hline(
        y=0,
        line_dash="dash",
        line_color="#000",
        opacity=0.5
    )
    
    # Добавляем финальную прибыль как аннотацию
    if ticker_data['profit_values']:
        final_profit = ticker_data['profit_values'][-1]
        fig.add_annotation(
            x=pd.to_datetime(ticker_data['profit_times'][-1]),
            y=final_profit,
            text=f"Final: ${final_profit:.2f}",
            showarrow=True,
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#764ba2",
            font=dict(size=12, color="#764ba2")
        )
    
    fig.update_layout(
        title=f'{ticker} - Cumulative Profit',
        xaxis_title='Time',
        yaxis_title='Profit ($)',
        template='plotly_white',
        height=300,
        showlegend=False
    )
    
    return fig

def fig_boundary_analysis(ticker_data, ticker):
    """Анализ динамических границ"""
    fig = go.Figure()
    
    # Анализ спреда границ
    if ticker_data['upper_boundary'] and ticker_data['lower_boundary']:
        min_len = min(len(ticker_data['upper_boundary']), len(ticker_data['lower_boundary']))
        spread = np.array(ticker_data['upper_boundary'][:min_len]) - np.array(ticker_data['lower_boundary'][:min_len])
        timestamps = pd.to_datetime(ticker_data['timestamps'][:min_len])
        
        fig.add_trace(go.Scatter(
            x=timestamps,
            y=spread,
            mode='lines',
            name='Boundary Spread',
            line=dict(color='#667eea', width=2),
            fill='tozeroy',
            fillcolor='rgba(102, 126, 234, 0.2)'
        ))
    
    # Относительные значения b_star и a_star
    if ticker_data['b_star_values'] and ticker_data['a_star_values']:
        min_len = min(len(ticker_data['b_star_values']), len(ticker_data['a_star_values']))
        b_star_timestamps = pd.to_datetime(ticker_data['timestamps'][:min_len])
        
        fig.add_trace(go.Scatter(
            x=b_star_timestamps,
            y=ticker_data['b_star_values'][:min_len],
            mode='lines',
            name='b* (Sell threshold)',
            line=dict(color='#ef553b', width=1.5, dash='dot'),
            opacity=0.7
        ))
        
        fig.add_trace(go.Scatter(
            x=b_star_timestamps,
            y=ticker_data['a_star_values'][:min_len],
            mode='lines',
            name='a* (Buy threshold)',
            line=dict(color='#00cc96', width=1.5, dash='dot'),
            opacity=0.7
        ))
    
    fig.update_layout(
        title=f'{ticker} - Boundary Analysis',
        xaxis_title='Time',
        yaxis_title='Value',
        template='plotly_white',
        height=300,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    return fig

def fig_final_profit_by_ticker(data):
    """Финальная прибыль по тикерам"""
    tickers = list(data.keys())
    profits = [data[ticker]['final_profit'] for ticker in tickers]
    num_trades = [data[ticker]['num_trades'] for ticker in tickers]
    num_buy = [data[ticker]['num_buy'] for ticker in tickers]
    num_sell = [data[ticker]['num_sell'] for ticker in tickers]
    
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=('Final Profit by Ticker', 'Number of Trades by Ticker'),
        vertical_spacing=0.15
    )
    
    # График прибыли
    colors = ['#00cc96' if p >= 0 else '#ef553b' for p in profits]
    fig.add_trace(
        go.Bar(
            x=tickers,
            y=profits,
            name='Final Profit',
            marker_color=colors,
            text=[f'${p:,.0f}' if abs(p) >= 1000 else f'${p:.0f}' for p in profits],
            textposition='auto'
        ),
        row=1, col=1
    )
    
    # График количества сделок
    fig.add_trace(
        go.Bar(
            x=tickers,
            y=num_buy,
            name='Buy Trades',
            marker_color='#4caf50',  # зеленый для покупок
            text=num_buy,
            textposition='inside'
        ),
        row=2, col=1
    )

    fig.add_trace(
        go.Bar(
            x=tickers,
            y=num_sell,
            name='Sell Trades',
            marker_color='#f44336',  # красный для продаж
            text=num_sell,
            textposition='inside'
        ),
        row=2, col=1
    )

    # Добавляем общее количество сделок над столбцами
    fig.add_trace(
        go.Scatter(
            x=tickers,
            y=num_trades,
            mode='text',
            text=num_trades,
            textposition='top center',
            showlegend=False,
            textfont=dict(size=12, color='black')
        ),
        row=2, col=1
    )

    fig.update_layout(
        barmode='group',  # группировка столбцов
        title_text="Number of Trades"
    )
    
    fig.update_layout(
        template='plotly_white',
        height=600,
        showlegend=False,
        margin=dict(t=50, b=50)
    )
    
    fig.update_yaxes(title_text='Profit ($)', row=1, col=1)
    fig.update_yaxes(title_text='Number of Trades', row=2, col=1)
    
    return fig

def fig_summary_statistics(data):
    """Сводная статистика"""
    tickers = list(data.keys())
    
    if not tickers:
        return go.Figure()
    
    # Рассчитываем агрегированные показатели
    total_profit = sum(data[ticker]['final_profit'] for ticker in tickers)
    avg_profit = total_profit / len(tickers)
    total_trades = sum(data[ticker]['num_trades'] for ticker in tickers)
    avg_trades = total_trades / len(tickers)
    final_profits = [data[ticker]['final_profit'] for ticker in tickers]
    avg_win_rate = sum(1 for p in final_profits if p > 0) / len(final_profits) * 100
    
    # Создаем индикаторы
    fig = go.Figure()
    
    fig.add_trace(go.Indicator(
        mode="number",
        value=total_profit,
        title={"text": "Total Profit"},
        number={'prefix': '$', 'valueformat': ',.0f'},
        domain={'row': 0, 'column': 0}
    ))
    
    fig.add_trace(go.Indicator(
        mode="number",
        value=avg_profit,
        title={"text": "Average Profit per Ticker"},
        number={'prefix': '$', 'valueformat': ',.0f'},
        domain={'row': 0, 'column': 1}
    ))
    
    fig.add_trace(go.Indicator(
        mode="number",
        value=total_trades,
        title={"text": "Total Trades"},
        number={'valueformat': ',.0f'},
        domain={'row': 1, 'column': 0}
    ))
    
    fig.add_trace(go.Indicator(
        mode="number",
        value=avg_win_rate,
        title={"text": "Average Win Rate"},
        number={'suffix': '%', 'valueformat': '.1f'},
        domain={'row': 1, 'column': 1}
    ))
    
    fig.update_layout(
        grid={'rows': 2, 'columns': 2, 'pattern': "independent"},
        template='plotly_white',
        height=300,
        margin=dict(t=30, b=30),
        title="Portfolio Summary"
    )
    
    return fig

def fig_trade_signals_table(ticker_data, ticker):
    """Таблица торговых сигналов для выбранного тикера"""
    signals = ticker_data.get('all_signals', [])
    
    if not signals:
        return go.Figure()
    
    # Создаем DataFrame из сигналов
    df = pd.DataFrame(signals, columns=['timestamp', 'price', 'action'])
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    # Рассчитываем P&L (если есть данные о прибылях)
    df['pnl'] = 0.0
    if ticker_data.get('all_profits'):
        profits_dict = dict(ticker_data['all_profits'])
        for idx, row in df.iterrows():
            if row['action'] == 'SELL':
                # Ищем соответствующую BUY транзакцию
                buy_idx = idx - 1
                while buy_idx >= 0 and df.loc[buy_idx, 'action'] != 'BUY':
                    buy_idx -= 1
                
                if buy_idx >= 0:
                    buy_price = df.loc[buy_idx, 'price']
                    sell_price = row['price']
                    df.loc[idx, 'pnl'] = sell_price - buy_price
    
    # Создаем таблицу
    fig = go.Figure(data=[go.Table(
        header=dict(
            values=['<b>Time</b>', '<b>Action</b>', '<b>Price ($)</b>', '<b>P&L ($)</b>'],
            fill_color='#667eea',
            align='center',
            font=dict(color='white', size=12)
        ),
        cells=dict(
            values=[
                df['timestamp'].dt.strftime('%Y-%m-%d %H:%M'),
                df['action'],
                df['price'].round(2),
                df['pnl'].round(2)
            ],
            fill=dict(color=['white', 'lightgrey']),
            align='center',
            font=dict(color='black', size=11),
            height=25
        )
    )])
    
    fig.update_layout(
        title=f'{ticker} - Trading Signals ({len(signals)} signals)',
        height=min(400, 50 + len(signals) * 30),
        margin=dict(t=50, b=20, l=20, r=20)
    )
    
    return fig

def fig_boundary_distribution(ticker_data, ticker):
    """Распределение граничных значений"""
    fig = go.Figure()
    
    # Гистограмма b_star_values
    if ticker_data['b_star_values']:
        fig.add_trace(go.Histogram(
            x=ticker_data['b_star_values'],
            name='b* (Sell)',
            marker_color='#ef553b',
            opacity=0.7,
            nbinsx=20
        ))
    
    # Гистограмма a_star_values
    if ticker_data['a_star_values']:
        fig.add_trace(go.Histogram(
            x=ticker_data['a_star_values'],
            name='a* (Buy)',
            marker_color='#00cc96',
            opacity=0.7,
            nbinsx=20
        ))
    
    fig.update_layout(
        title=f'{ticker} - Boundary Value Distribution',
        xaxis_title='Boundary Value',
        yaxis_title='Frequency',
        template='plotly_white',
        height=300,
        barmode='overlay',
        bargap=0.1,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    
    return fig

# ==============================================================
# 3. ФУНКЦИЯ ДЛЯ СОЗДАНИЯ ДАШБОРДА
# ==============================================================
def build_trading_dashboard(results):
    """Создание дашборда Dash из results"""
    
    # Обрабатываем данные из results
    data = extract_data_from_results(results)
    tickers = list(data.keys())
    
    if not tickers:
        print("No data available in results!")
        return None
    
    # Создаем приложение Dash
    app = dash.Dash(__name__)
    
    app.layout = html.Div([
        # Заголовок
        html.H1(
            "Dynamic Optimal Stopping Dashboard",
            style={
                'textAlign': 'center',
                'color': '#2c3e50',
                'marginBottom': '30px',
                'paddingTop': '20px',
                'fontSize': '32px'
            }
        ),
        
        html.H3(
            "Analysis of Trading Strategies with optimal stopping",
            style={
                'textAlign': 'center',
                'color': '#7f8c8d',
                'marginBottom': '40px',
                'fontSize': '18px'
            }
        ),
        
        # Панель управления
        html.Div([
            html.Label("Select Ticker:", 
                      style={'fontWeight': 'bold', 'marginRight': '10px', 'fontSize': '14px'}),
            dcc.Dropdown(
                id='ticker-dropdown',
                options=[{'label': ticker, 'value': ticker} for ticker in tickers],
                value=tickers[0],
                style={'width': '200px', 'display': 'inline-block', 'marginRight': '30px'}
            ),
            
            html.Label("Analysis Type:", 
                      style={'fontWeight': 'bold', 'marginRight': '10px', 'fontSize': '14px'}),
            dcc.Dropdown(
                id='analysis-type-dropdown',
                options=[
                    {'label': 'Price Analysis', 'value': 'price'},
                    {'label': 'Profit Analysis', 'value': 'profit'},
                    {'label': 'Signal Analysis', 'value': 'signal'}
                ],
                value='price',
                style={'width': '200px', 'display': 'inline-block'}
            ),
        ], style={
            'backgroundColor': '#f8f9fa',
            'padding': '20px',
            'borderRadius': '10px',
            'marginBottom': '30px',
            'border': '1px solid #e9ecef'
        }),
        
        # Общие показатели (ряд 1)
        html.Div([
            dcc.Graph(
                id='summary-stats',
                figure=fig_summary_statistics(data),
                style={'height': '100%'}
            )
        ], style={'width': '100%', 'marginBottom': '30px'}),
        
        # Финансовые показатели по тикерам (ряд 2)
        html.Div([
            dcc.Graph(
                id='portfolio-overview',
                figure=fig_final_profit_by_ticker(data),
                style={'height': '100%'}
            )
        ], style={'width': '100%', 'marginBottom': '30px'}),
        
        # Основной график для выбранного тикера
        html.Div([
            html.H3(id='selected-ticker-title', 
                   style={'textAlign': 'center', 'marginBottom': '20px', 'color': '#34495e'}),
            
            dcc.Graph(id='main-chart', style={'height': '500px'})
        ], style={'width': '100%', 'marginBottom': '30px'}),
        
        # Вспомогательные графики (ряд 4)
        html.Div([
            html.Div([
                dcc.Graph(id='profit-chart', style={'height': '100%'})
            ], style={'width': '100%', 'display': 'inline-block', 'height': '350px'}),
        ], style={'width': '100%', 'marginBottom': '30px'}),
        
        # Таблица сигналов (ряд 5)
        html.Div([
            dcc.Graph(id='signals-table')
        ], style={'width': '100%', 'marginBottom': '30px'}),
        
        # Информация в футере
        html.Div([
            html.P(
                f"Dashboard generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} | "
                f"Total tickers analyzed: {len(tickers)} | "
                f"Total signals: {sum(len(data[t]['all_signals']) for t in tickers)}",
                style={'textAlign': 'center', 'color': '#7f8c8d', 'fontSize': '12px'}
            )
        ], style={
            'marginTop': '40px',
            'paddingTop': '20px',
            'borderTop': '1px solid #ecf0f1',
            'backgroundColor': '#f8f9fa',
            'padding': '15px',
            'borderRadius': '5px'
        })
    ], style={
        'backgroundColor': 'white',
        'padding': '20px',
        'maxWidth': '1400px',
        'margin': '0 auto',
        'fontFamily': 'Arial, sans-serif'
    })
    
    # Callback для обновления всех графиков
    @app.callback(
        [Output('main-chart', 'figure'),
         Output('profit-chart', 'figure'),
         Output('signals-table', 'figure'),
         Output('selected-ticker-title', 'children')],
        [Input('ticker-dropdown', 'value'),
         Input('analysis-type-dropdown', 'value')]
    )
    def update_all_charts(selected_ticker, analysis_type):
        ticker_data = data[selected_ticker]
        
        # Основной график в зависимости от типа анализа
        if analysis_type == 'price':
            main_fig = fig_price_with_boundaries(ticker_data, selected_ticker)
        elif analysis_type == 'profit':
            main_fig = fig_cumulative_profit(ticker_data, selected_ticker)
        else:  # 'signal'
            main_fig = fig_trade_signals_table(ticker_data, selected_ticker)
        
        # Вспомогательные графики
        profit_fig = fig_cumulative_profit(ticker_data, selected_ticker)
        table_fig = fig_trade_signals_table(ticker_data, selected_ticker)
        
        # Заголовок
        titles = {
            'price': f"{selected_ticker} - Price Analysis with Dynamic Boundaries",
            'profit': f"{selected_ticker} - Profit Analysis",
            'boundary': f"{selected_ticker} - Boundary Analysis",
            'signal': f"{selected_ticker} - Signal Analysis"
        }
        title = titles.get(analysis_type, f"{selected_ticker} Analysis")
        
        return main_fig, profit_fig, table_fig, title
    
    # Callback для обновления сводной статистики
    @app.callback(
        Output('summary-stats', 'figure'),
        [Input('ticker-dropdown', 'value')]
    )
    def update_summary(selected_ticker):
        return fig_summary_statistics(data)
        
    
    return app

# ==============================================================
# 4. ФУНКЦИЯ ДЛЯ ЗАПУСКА
# ==============================================================
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import pandas as pd
from datetime import datetime
import plotly.io as pio

def build_trading_dashboard_1(results, save_html=True, filename="trading_dashboard.html"):
    """
    Создание дашборда из results и сохранение в HTML файл
    
    Args:
        results: результаты анализа
        save_html: сохранять ли в HTML файл
        filename: имя файла для сохранения
    """
    
    # Обрабатываем данные из results (замените на вашу реализацию)
    data = extract_data_from_results(results)
    tickers = list(data.keys())
    
    if not tickers:
        print("No data available in results!")
        return None
    
    # 1. СОЗДАЕМ ВСЕ СТАТИЧЕСКИЕ ГРАФИКИ
    all_figures = {}
    
    # Создаем все графики для каждого тикера и типа анализа
    for ticker in tickers:
        ticker_data = data[ticker]
        
        all_figures[ticker] = {
            'price': fig_price_with_boundaries(ticker_data, ticker),
            'profit': fig_cumulative_profit(ticker_data, ticker),
            'signal': fig_trade_signals_table(ticker_data, ticker),
            'summary': fig_summary_statistics(data),
            'portfolio': fig_final_profit_by_ticker(data)
        }
    
    # 2. СОЗДАЕМ HTML С ДИНАМИЧЕСКИМ ИНТЕРФЕЙСОМ НА JavaScript
    html_content = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Trading Dashboard</title>
        <script src="https://cdn.plot.ly/plotly-2.27.0.min.js"></script>
        <script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.6.0/jquery.min.js"></script>
        <style>
            body {{
                font-family: Arial, sans-serif;
                margin: 0;
                padding: 20px;
                background-color: #f5f5f5;
                max-width: 1400px;
                margin: 0 auto;
            }}
            
            .header {{
                text-align: center;
                color: #2c3e50;
                margin-bottom: 30px;
                padding-top: 20px;
                font-size: 32px;
            }}
            
            .subheader {{
                text-align: center;
                color: #7f8c8d;
                margin-bottom: 40px;
                font-size: 18px;
            }}
            
            .control-panel {{
                background-color: #f8f9fa;
                padding: 20px;
                border-radius: 10px;
                margin-bottom: 30px;
                border: 1px solid #e9ecef;
            }}
            
            .control-group {{
                display: inline-block;
                margin-right: 30px;
            }}
            
            label {{
                font-weight: bold;
                margin-right: 10px;
                font-size: 14px;
            }}
            
            select {{
                width: 200px;
                padding: 8px;
                border-radius: 5px;
                border: 1px solid #ddd;
                font-size: 14px;
            }}
            
            .chart-container {{
                margin-bottom: 30px;
                background: white;
                padding: 15px;
                border-radius: 10px;
                box-shadow: 0 2px 10px rgba(0,0,0,0.1);
            }}
            
            .chart-title {{
                text-align: center;
                margin-bottom: 20px;
                color: #34495e;
                font-size: 20px;
            }}
            
            .grid-container {{
                display: grid;
                grid-template-columns: 1fr;
                gap: 30px;
            }}
            
            .footer {{
                margin-top: 40px;
                padding-top: 20px;
                border-top: 1px solid #ecf0f1;
                background-color: #f8f9fa;
                padding: 15px;
                border-radius: 5px;
                text-align: center;
                color: #7f8c8d;
                font-size: 12px;
            }}
            
            #main-chart, #profit-chart, #signals-table {{
                height: 500px;
            }}
            
            #summary-stats, #portfolio-overview {{
                height: 400px;
            }}
        </style>
    </head>
    <body>
        <h1 class="header">Dynamic Optimal Stopping Dashboard</h1>
        <h3 class="subheader">Analysis of Trading Strategies with optimal stopping</h3>
        
        <div class="control-panel">
            <div class="control-group">
                <label for="ticker-select">Select Ticker:</label>
                <select id="ticker-select" onchange="updateDashboard()">
                    {''.join([f'<option value="{ticker}">{ticker}</option>' for ticker in tickers])}
                </select>
            </div>
            
            <div class="control-group">
                <label for="analysis-select">Analysis Type:</label>
                <select id="analysis-select" onchange="updateDashboard()">
                    <option value="price">Price Analysis</option>
                    <option value="profit">Profit Analysis</option>
                    <option value="signal">Signal Analysis</option>
                </select>
            </div>
        </div>
        
        <div class="grid-container">
            <!-- Сводная статистика -->
            <div class="chart-container">
                <div id="summary-stats"></div>
            </div>
            
            <!-- Обзор портфеля -->
            <div class="chart-container">
                <div id="portfolio-overview"></div>
            </div>
            
            <!-- Основной график -->
            <div class="chart-container">
                <div id="selected-ticker-title" class="chart-title"></div>
                <div id="main-chart"></div>
            </div>
            
            <!-- График прибыли -->
            <div class="chart-container">
                <div id="profit-chart"></div>
            </div>
            
            <!-- Таблица сигналов -->
            <div class="chart-container">
                <div id="signals-table"></div>
            </div>
        </div>
        
        <div class="footer">
            Dashboard generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')} | 
            Total tickers analyzed: {len(tickers)} | 
            Total signals: {sum(len(data[t]['all_signals']) for t in tickers)}
        </div>
        
        <script>
            // Храним все графики
            const allFigures = {{
                {','.join([f'"{ticker}": ' + '{' + 
                  f'"price": {all_figures[ticker]["price"].to_json()}, ' +
                  f'"profit": {all_figures[ticker]["profit"].to_json()}, ' +
                  f'"signal": {all_figures[ticker]["signal"].to_json()}' + '}' 
                  for ticker in tickers])}
            }};
            
            const summaryFigure = {all_figures[tickers[0]]['summary'].to_json()};
            const portfolioFigure = {all_figures[tickers[0]]['portfolio'].to_json()};
            
            function updateDashboard() {{
                const selectedTicker = document.getElementById('ticker-select').value;
                const analysisType = document.getElementById('analysis-select').value;
                
                // Обновляем заголовок
                const titles = {{
                    'price': `${{selectedTicker}} - Price Analysis with Dynamic Boundaries`,
                    'profit': `${{selectedTicker}} - Profit Analysis`,
                    'signal': `${{selectedTicker}} - Signal Analysis`
                }};
                document.getElementById('selected-ticker-title').textContent = 
                    titles[analysisType] || `${{selectedTicker}} Analysis`;
                
                // Обновляем графики
                Plotly.react('main-chart', allFigures[selectedTicker][analysisType].data, allFigures[selectedTicker][analysisType].layout);
                Plotly.react('profit-chart', allFigures[selectedTicker]['profit'].data, allFigures[selectedTicker]['profit'].layout);
                Plotly.react('signals-table', allFigures[selectedTicker]['signal'].data, allFigures[selectedTicker]['signal'].layout);
                Plotly.react('summary-stats', summaryFigure.data, summaryFigure.layout);
                Plotly.react('portfolio-overview', portfolioFigure.data, portfolioFigure.layout);
            }}
            
            // Инициализация при загрузке
            document.addEventListener('DOMContentLoaded', function() {{
                updateDashboard();
            }});
        </script>
    </body>
    </html>
    """
    
    # 3. СОХРАНЯЕМ В ФАЙЛ
    if save_html:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(html_content)
        print(f"Dashboard saved to {filename}")
    
    return html_content


def run_dashboard(results, port=8050):
    """
    Запускает дашборд с данными из results
    
    Parameters:
    -----------
    results : dict
        Словарь с результатами торговли по тикерам
    port : int
        Порт для запуска дашборда (по умолчанию 8050)
    """
    print("Building dashboard from results data...")
    
    # Строим дашборд
    app = build_trading_dashboard(results)
    build_trading_dashboard_1(results)
        
    if app is None:
        print("Failed to build dashboard. Check your results data.")
        return
    
    # Запускаем сервер
    print(f"Dashboard is running at http://127.0.0.1:{port}")
    print("Press Ctrl+C to stop")
    
    try:
        app.run(debug=True)
    except Exception as e:
        print(f"Error running dashboard: {e}")

# ==============================================================
# 6. MAIN EXECUTION
# ==============================================================
def main():
    """Main execution function."""
    print("\n" + "=" * 60)
    print("DYNAMIC OPTIMAL STOPPING STRATEGY WITH BOUNDARY PLOTS")
    print("=" * 60)
    
    if not price_data:
        print("No data available. Exiting.")
        return
    
    results = {}
    
    for ticker, price_series in price_data.items():
        print(f"\nProcessing {ticker}...")
        
        # Initialize trader
        trader = DynamicOptimalStoppingTrader(
            rho=0.5,      # Discount rate
            kappa=5.0,    # Strike/cost
            theta=0.5,    # Profit exponent
            lam=0.1       # Temperature
        )
        
        # Process each tick
        price_values = []
        ratio = 1.0
        df = pd.DataFrame(price_data[ticker].reset_index())
        for i, (timestamp, price) in enumerate(zip(df['Datetime'], df[ticker])):
            
            price_values.append(price)
            price_history = pd.Series(price_values)
            
            # Process tick
            action, current_price, ratio = trader.process_tick(
                timestamp, price, price_history
            )
            
            # Progress indicator
            if i % 100 == 0 and i > 0:
                print(f"  Processed {i} ticks, last action: {action}")
                if trader.b_star is not None:
                    print(f"    Current boundaries: buy={trader.a_star:.3f}, sell={trader.b_star:.3f}")
        
        # Calculate profits
        trader.calculate_profits()
        
        # Store results
        summary = trader.get_summary()
        results[ticker] = {
            'trader': trader,
            'signals': trader.signals,
            'profits': trader.profits,
            'summary': summary,
            'boundary_history': trader.boundary_history
        }
        
        print(f"  Total signals: {summary['num_signals']}")
        print(f"  Final profit: ${summary['final_profit']:.2f}")
        if trader.b_star is not None:
            print(f"  Final boundaries: buy={trader.a_star:.3f}, sell={trader.b_star:.3f}")
        print(f"  Boundary records: {len(trader.boundary_history['timestamps'])}")        
    
    # Summary dashboard
    if results:
        print("\n" + "=" * 60)
        print("CREATING SUMMARY DASHBOARD")
        print("=" * 60)
        run_dashboard(results)
        
        # Plot 1: Final profits
        tickers_list = list(results.keys())
        final_profits = [results[t]['summary']['final_profit'] for t in tickers_list]
        num_trades = [results[t]['summary']['num_signals'] for t in tickers_list]
                
        # Statistics
        total_profit = sum(final_profits)
        avg_profit = np.mean(final_profits)
        win_rate = sum(1 for p in final_profits if p > 0) / len(final_profits) * 100
        
        print(f"\nSUMMARY STATISTICS:")
        print(f"Total Profit: ${total_profit:.2f}")
        print(f"Average Profit: ${avg_profit:.2f}")
        print(f"Win Rate: {win_rate:.1f}%")
        print(f"Total Trades: {sum(num_trades)}")

# ==============================================================
# 7. RUN MAIN FUNCTION
# ==============================================================
if __name__ == "__main__":
    main()

DOWNLOADING STOCK DATA
✓ Downloaded AAPL: 1946 rows
✓ Downloaded MSFT: 1946 rows
✓ Downloaded GOOGL: 1946 rows
✓ Downloaded AMZN: 1946 rows
✓ Downloaded META: 1946 rows
✓ Downloaded NVDA: 1946 rows
✓ Downloaded BRK-B: 1946 rows
✓ Downloaded JPM: 1946 rows
✓ Downloaded V: 1946 rows
✓ Downloaded JNJ: 1946 rows
✓ Downloaded WMT: 1946 rows
✓ Downloaded PG: 1946 rows
✓ Downloaded HD: 1946 rows
✓ Downloaded BAC: 1946 rows
✓ Downloaded DIS: 1946 rows
✓ Downloaded ADBE: 1946 rows
✓ Downloaded PFE: 1946 rows
✓ Downloaded NFLX: 1946 rows

DYNAMIC OPTIMAL STOPPING STRATEGY WITH BOUNDARY PLOTS

Processing AAPL...
  Processed 100 ticks, last action: BUY
    Current boundaries: buy=0.999, sell=1.001
  Processed 200 ticks, last action: HOLD
    Current boundaries: buy=0.999, sell=1.001
  Processed 300 ticks, last action: HOLD
    Current boundaries: buy=0.999, sell=1.001
  Processed 400 ticks, last action: BUY
    Current boundaries: buy=0.999, sell=1.001
  Processed 500 ticks, last action: HOLD
    


SUMMARY STATISTICS:
Total Profit: $457.10
Average Profit: $25.39
Win Rate: 77.8%
Total Trades: 5420
